## Code Start to Add concrete grade to all Piles

In [2]:
import pandas as pd
import numpy as np
import random
import ifcopenshell
import ifcopenshell.util.element
from ifcopenshell.util.selector import Selector
from ifcopenshell.util.element import get_psets as check

from ifcopenshell.util.placement import get_local_placement
import datetime
import random
from ifcopenshell.util.selector import Selector
import uuid

create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)

## File name path here 
ifc_file= ifcopenshell.open('OMC2-R8.ifc')

## Define elements to select
Piles=ifc_file.by_type("IfcPile")

from ifcopenshell.util.element import get_psets as check

In [3]:
def random_grade():
    return random.choice(["G35","G40","G45"])

In [4]:
random_grade()

'G35'

In [4]:


## Create concrete grade properties - Properties name = Concrete Grade, Value = (random grade, field type = IfcText)
## Concrete_grade_prop =ifc_file.createIfcPropertySingleValue("Concrete Grade", "Concrete Grade", ifc_file.create_entity("IfcText", random_grade()), None)

## required - used the same owner history as ifc_file
owner_history = ifc_file.by_type("IfcOwnerHistory")[0]

## Loop thru elements in Piles
for each in Piles:
    ## For each isdefined by
    for each2 in each.IsDefinedBy:
        ## If there is "Materials and Finishes" properties
        if (each2.RelatingPropertyDefinition.Name=="Materials and Finishes"):
            
            ## Concat (Add) Concrete grade properties to Materials and Finishes properties set
            each2.RelatingPropertyDefinition.HasProperties+=tuple([ifc_file.createIfcPropertySingleValue("Concrete Grade", "Concrete Grade", ifc_file.create_entity("IfcText", random_grade()), None)])
            print(each2.RelatingPropertyDefinition.HasProperties)
            break;
   
        else:
            ## Create Materials and finishes property set , and also assign concrete grade properties to it
            property_set = ifc_file.createIfcPropertySet(create_guid(), owner_history, "Materials and Finishes", None, [ifc_file.createIfcPropertySingleValue("Concrete Grade", "Concrete Grade", ifc_file.create_entity("IfcText", random_grade()), None)])
            
            ## Create relationship between the element and the properties set that we just created, and assigning a unique guid for the relationship
            ifc_file.createIfcRelDefinesByProperties(create_guid(), owner_history, None, None, [each], property_set)
            break;


## DO NOT RUN MORE THAN 1 TIMES , IF NOT THERE WILL BE MORE THAN 1 CONCRETE GRADE PROPERTIES IN MATERIALS AND FINISHES

## To improve: add a check to see if we already have cocnrete grade in material and finshes

In [5]:
ifc_file_after= 'OMC2_R8_Mod_R1.ifc'
ifc_file.write(ifc_file_after)

## Code Completed to add concrete grade to all Piles

## Code Start to add construction type for structural elements

In [16]:
## To add construction type to Materials and finishes properties set from either Precast or Cast in situ

def random_con_type():
    return random.choice(["Precast","Cast in situ"])

In [17]:
selector=Selector()

## Select all structural element
Strctural_ele=selector.parse(ifc_file, '.IfcWall | .IfcColumn | .IfcBeam | .IfcSlab | .IfcStair  ')


In [20]:
## Loop thru the elements
for each in Strctural_ele:
    ## If Z value >0 - means not pile
    if (get_local_placement(each.ObjectPlacement)[0:3,3][2])>0:
        for each2 in each.IsDefinedBy:
            ## Check if Materials and Finishes properties exsit
            if (each2.RelatingPropertyDefinition.Name=="Materials and Finishes"):
                
                ## Concat (Add) Concrete grade properties to Materials and Finishes properties set
                each2.RelatingPropertyDefinition.HasProperties+=tuple([ifc_file.createIfcPropertySingleValue("Construction Type", "Construction Type", ifc_file.create_entity("IfcText", random_con_type()), None)])
                print(each2.RelatingPropertyDefinition.HasProperties)
                break;

            else:
                ## Create Materials and finishes property set , and also assign concrete grade properties to it
                property_set = ifc_file.createIfcPropertySet(create_guid(), owner_history, "Materials and Finishes", None, [ifc_file.createIfcPropertySingleValue("Construction Type", "Construction Type", ifc_file.create_entity("IfcText", random_con_type()), None)])
               
                ## Create relationship between the element and the properties set that we just created, and assigning a unique guid for the relationship
                ifc_file.createIfcRelDefinesByProperties(create_guid(), owner_history, None, None, [each], property_set)
                break;
    else:
        pass
    
## DO NOT RUN MORE THAN 1 TIMES , same as above

## To improve: SAME AS ABOVE

In [25]:
ifc_file_after= 'OMC2_R8_Mod_R1.ifc'
ifc_file.write(ifc_file_after)

## Code Complete to add construction type for structural elements

## Code Start for Structural Type

In [89]:
## Add IFC export type too all select elements, under text properties set

selector=Selector()
Strctural_ele=selector.parse(ifc_file, '.IfcWall | .IfcColumn | .IfcBeam | .IfcSlab | .IfcStair  ')

len(Strctural_ele)

1186

In [90]:
IFCExportType_prop =ifc_file.createIfcPropertySingleValue("IFCExportType", "IFCExportType", ifc_file.create_entity("IfcText", ""), None)


for each in Strctural_ele:
    for each2 in each.IsTypedBy[0].RelatingType.HasPropertySets:
        if each2.Name=="Text":
            for each3 in each2.HasProperties:
                if each3.Name=="IfcExportType":
                    break
        else:
            property_set = ifc_file.createIfcPropertySet(create_guid(), owner_history, "Text", None, [ifc_file.createIfcPropertySingleValue("IFCExportType", "IFCExportType", ifc_file.create_entity("IfcText", ""), None)])
            ifc_file.createIfcRelDefinesByProperties(create_guid(), owner_history, None, None, [each], property_set)
            break;

            
                

In [45]:
ifc_file_after= 'OMC2_R8_After_Products.ifc'
ifc_file.write(ifc_file_after)

## Code Completed for Structural Type

## Code Start  to Add concrete grade to all structural elements where Z > 0

In [10]:

## Select all structural element to add concrete grade
selector=Selector()
Strctural_concrete_grade=selector.parse(ifc_file, '.IfcWall | .IfcColumn | .IfcBeam | .IfcSlab | .IfcStair  ')


In [17]:

element=Strctural_concrete_grade

## Select all structural element to add concrete grade
## property_values =ifc_file.createIfcPropertySingleValue("Concrete Grade", "Concrete Grade", ifc_file.create_entity("IfcText", "G45"), None)

owner_history = ifc_file.by_type("IfcOwnerHistory")[0]


for each in element:
    try:
        ## Check if Concrete grade is available already in material properpeties set ( Its a diffrent properties set than the direct relationship - if an element belongs to a material group which have concrete grade, it applies to the element also)
        if "Concrete Grade" in str(test_set[9].HasAssociations[0].RelatingMaterial.ForLayerSet.MaterialLayers):
            break;
        else:
            pass
    except:
        pass
    
    ## Loop thry elements properties
    for each2 in each.IsDefinedBy:
        ## Check if materials and finishes is in it
        if "Materials and Finishes" not in list(each2.RelatingPropertyDefinition):
            ## Create new property set, with concrete grade G45
            property_set = ifc_file.createIfcPropertySet(create_guid(), owner_history, "Materials and Finishes", None, [ifc_file.createIfcPropertySingleValue("Concrete Grade", "Concrete Grade", ifc_file.create_entity("IfcText", "G45"), None)])
            
            ## Create relationship between the elements and the property set
            ifc_file.createIfcRelDefinesByProperties(create_guid(), owner_history, None, None, [each], property_set)
            break;
        
        ## Check if conrete grade is available or not - instead of checking materials and finishes - avoid duplicate , but wont work if other properties set have Concrete grade   
        elif "Concrete Grade" not in list(each2.RelatingPropertyDefinition.HasProperties):
       
            each2.RelatingPropertyDefinition.HasProperties+=tuple([ifc_file.createIfcPropertySingleValue("Concrete Grade", "Concrete Grade", ifc_file.create_entity("IfcText", "G45"), None)])
            break;
            
            
## CAN RUN MORE THAN 1 TIMES , BUT CODE DOESNT WORK IF THERE ARE MORE THAN 1 CONCRETE GRADE


In [16]:
ifc_file_after= 'OMC2_R8_Mod_R1.ifc'
ifc_file.write(ifc_file_after)

## Code complete to Add concrete grade to all structural elements where Z > 0

## Code start for adding Type of Constrction, Type of Wall and post tendon to Z>0

In [22]:
selector=Selector()
Strctural=selector.parse(ifc_file, '.IfcWall | .IfcColumn | .IfcBeam | .IfcSlab | .IfcStair  ')


In [49]:
len(Strctural)

1186

In [67]:
for each in Strctural:
    if(get_local_placement(each.ObjectPlacement)[0:3,3][2])<0:
        Strctural.remove(each)


In [51]:
Strctural[0].is_a('IfcPile')

False

In [52]:
def random_method():
    return random.choice(["Precast","Cast in-situ"])

for each in element:
    if type of contruction not exist/empty:
        create random precast/Cast In situ
    
    if each is a Wall:
        if type of wall not exisit or empty 
            - random type of wall
        else 
            pass
    if each is a beam or slab
             if type of construction  == Precast
                 if post tension not existim empty
                         create post tendion ratio
   

In [ ]:

## NOT COMPLETE YET

## Code complete for adding Type of Constrction, Type of Wall and post tendon to Z>0